In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [6]:
class MTS:   
    def __init__(self, ts):
             self.ts = ts
            
    def cov_mat(self, centering = True):
        stdsc = StandardScaler()
        X = self.ts
        X = stdsc.fit_transform(X)
        self.ts = X
        return X.transpose() @ X

In [3]:
class CPCA:
    def __init__(self, epsilon = 1e-5):
        self.cov = None
        self.epsilon = epsilon
        self.U = None
        self.V = None
        self.S = None
    
    def fit(self,listMTS):
        if (len(listMTS) > 0):
            P = listMTS[0].cov_mat().shape[1]
            cov_mat = [mat.cov_mat() for mat in listMTS]
            self.cov = sum(cov_mat)/len(cov_mat)
            #Add epsilon Id in order to ensure invertibility
            cov = self.cov + self.epsilon*np.eye(P)
            #Compute SVD
            U,S,V = np.linalg.svd(self.cov)
            #Save SVD
            self.U = U
            self.S = S
            self.V = V
        

    def pred(self, listMTS, ncp):
        predicted = []
        if (self.U is not None):
            predicted = [elem.ts @ self.U[:,:ncp] for elem in listMTS]
        return predicted
    
    def reconstitution_error(self, listMTS, ncp):
        mse = np.full(len(listMTS),np.inf)
        if (self.U is not None):
            prediction = self.pred(listMTS, ncp)
            reconstit = [elem @ ((self.U)[:,:ncp].transpose()) for elem in prediction]
            mse = [((listMTS[i].ts - reconstit[i])**2).sum() for i in range(len(prediction))]
        return mse

## MTS

In [7]:
#Import lp1 for test
import pandas as pd
res = [pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data", sep = "\t",skiprows=1+(18*i), nrows=15, header = None) for i in range(1,80)]
res = [MTS(elem.drop(columns = [0]).to_numpy()) for elem in res]

In [9]:
res[0].ts

array([[ -1,  -1,  63,  -2,  -1,   0],
       [ -1,  -1,  63,  -3,  -1,   0],
       [ -1,  -1,  61,  -3,   0,   0],
       [  0,  -4,  63,   1,   0,   0],
       [  0,  -1,  59,  -2,   0,  -1],
       [ -3,   3,  57,  -8,  -3,  -1],
       [ -1,   3,  70, -10,  -2,  -1],
       [  0,  -3,  61,   0,   0,   0],
       [  0,  -2,  53,  -1,  -2,   0],
       [  0,  -3,  66,   1,   4,   0],
       [ -3,   3,  58, -10,  -5,   0],
       [ -1,  -1,  66,  -4,  -2,   0],
       [ -1,  -2,  67,  -3,  -1,   0],
       [  0,   1,  66,  -6,  -3,  -1],
       [ -1,  -1,  59,  -3,  -4,   0]])

In [5]:
name = [pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data", sep = "\t",skiprows=(18*i), nrows=1, header = None) for i in range(1,80)]

In [6]:
name = [elem[0][0] for elem in name]

name_unique = list(np.unique(name))
dict_name = dict(zip(name_unique, list(range(len(name_unique)))))

gt_nb_cluster = np.array([dict_name.get(nom) for nom in name])

$$Pre $=\sum_{j=1}^{K} \underbrace{\frac{\left|C_{j}\right|}{N}}_{\text{prop_part}} \times \underbrace{\max _{i=1,2, \cdots, g} \frac{\left|G_{i} \cap C_{j}\right|}{\left|C_{j}\right|}}_{\text{max_part}}$$

In [85]:
class Mc2PCA:
    def __init__(self,K, ncp, itermax = 1000, conv_crit = 1e-5):
        self.K = K
        self.N = None
        self.ncp = ncp
        self.iter_max = itermax
        self.converged = False
        self.CPCA_final = None
        self.conv_crit = conv_crit
        self.pred = None
        
    def fit(self, X):
        N = len(X)
        #initialisation
        index_cluster = np.tile(np.arange(self.K), int(N/self.K) + 1)[:N]
        to_continue = True
        i = 0
        old_error = -1
        
        while to_continue:

            #Split all MTS according to the cluster 
            #we store it in a list of lists of MTS (each list inside the list corresponding to a cluster)
            MTS_by_cluster = [[X[i] for i in list(np.where(index_cluster == j)[0])] for j in range(self.K)]

            CPCA_by_cluster = [CPCA() for i in range(self.K)]

            #fit by cluster
            [CPCA_by_cluster[i].fit(MTS_by_cluster[i]) for i in range(self.K)]

            res = np.array([cpca.reconstitution_error(X, self.ncp) for cpca in CPCA_by_cluster])
            #Update index cluster
            index_cluster = res.argmin(axis = 0)

            #new total error 
            new_error = res.min(axis = 0).sum()
            to_continue = (abs(old_error - new_error) > self.conv_crit) & (self.iter_max > i)
            self.converged = np.abs(old_error - new_error) < self.conv_crit

            #Updata
            old_error = new_error 
            i += 1
        self.CPCA_final = CPCA_by_cluster
        self.pred = index_cluster
        return index_cluster
    
    def precision(self,gt_cluster):
        index_cluster = self.pred
        N = gt_cluster.shape[0]
        g = np.unique(gt_cluster)
        nb_g = g.shape[0]

        G = [np.where(gt_cluster == i)[0] for i in range(nb_g)]
        C = [np.where(index_cluster == i)[0] for i in range(self.K)]
        
        #to handle case where a cluster is empty
        max_part = list()
        for j in range(self.K):
            l = list()
            for i in range(nb_g):
                if len(C[j])!=0:
                    l.append([np.intersect1d(G[i],C[j]).shape[0]/C[j].shape[0]])
                else:
                    l.append(0)
            max_part.append(np.max(l))
        max_part = np.array(max_part)
        
        #max_part = np.array([max([np.intersect1d(G[i],C[j]).shape[0]/C[j].shape[0] for i in range(nb_g)]) for j in range(self.K)])
        prop_part = np.array([C[j].shape[0]/N for j in range(self.K)])
        return max_part.dot(prop_part)

In [103]:
m = Mc2PCA(4,4)
m.fit(res)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 1,
       3, 3, 1, 3, 1, 3, 3, 3, 3, 0, 0, 0, 2, 3, 0, 0, 1, 3, 3, 3, 0, 3,
       3, 2, 2, 2, 0, 1, 1, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 0, 3, 3, 1, 3,
       3, 1, 1, 0, 0, 2, 2, 0, 2, 3, 2, 2, 3])

In [104]:
m.precision(gt_nb_cluster)

0.5189873417721519

### Search for best parameter ncp

In [101]:
ncp_list = np.arange(1,8)

def search_ncp(X,K,ncp_list,y_true):
    pres = np.zeros(ncp_list.shape[0])
    for i in range(len(ncp_list)):
        m = Mc2PCA(K,ncp_list[i])
        m.fit(X)
        pres[i] = m.precision(y_true)
    pre = np.max(pres)
    best_ncp = ncp_list[np.argmax(pres)]
    return best_ncp, pre

In [102]:
search_ncp(res,4,ncp_list,gt_nb_cluster)

(6, 0.5443037974683544)